In [1]:
import glob
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [2]:
import tensorflow as tf

print(tf.__version__)

2.4.0


# Show test image and mask

In [3]:
def img_show(img_path, mask_path, model):
    img = load_img(img_path, color_mode = "grayscale")
    mask = load_img(mask_path, color_mode = "grayscale")
    
    origin = cv2.imread(img_path,cv2.IMREAD_COLOR)
    origin = cv2.resize(origin, (512, 512))
    
    
    img = img.resize((512,512))
    mask = mask.resize((512,512))
    
    img = img_to_array(img)
    mask = img_to_array(mask)
    
    img.astype('float32')
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    pred = model.predict(img)
    
    pred[pred > 0.5] = 255
    pred[pred <= 0.5] = 0
    
    
    cv2.imshow("image",origin)
    cv2.imshow("pred",pred[0])
    cv2.imshow("label",mask)
    
    # a를 누르면 종료
    if( cv2.waitKey() == ord('a')):
        cv2.destroyAllWindows()
        return 1
    
    return 0

# Show test vedio

In [4]:
def video_show(video_file, model):
    cap = cv2.VideoCapture(video_file)

    while(cap.isOpened()):
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (512, 512))

        img = np.float32(frame)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)
    
        pred = model.predict(img)

        pred[pred > 0.5] = 255
        pred[pred <= 0.5] = 0
    
        cv2.imshow("img",frame)
        cv2.imshow("pred",pred[0])

        if cv2.waitKey(1) & 0xFF == ord('a'):
            break
    cv2.destroyAllWindows()
    cap.release()

# Model

In [8]:
def dice_coef(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred)

def sens(y_true, y_pred): # sensitivity, recall
    print(y_pred)
    print(y_true)
    y_target_yn = K.round(K.clip(y_true, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall

In [9]:
model = load_model('C:/Users/s_wnsgk4041/crack_segmentation_dataset/crack_check/final_15_0.150833.hdf5',
                                       custom_objects={'sens':sens,'dice_coef_loss': dice_coef_loss})

## Image show example

In [11]:
img_path = sorted(glob.glob("C:/Users/s_wnsgk4041/crack_segmentation_dataset/test/images/*.jpg"))
mask_path = sorted(glob.glob("C:/Users/s_wnsgk4041/crack_segmentation_dataset/test/masks/*.jpg"))

for i in zip(img_path,mask_path):
    if (os.path.split(i[0])[1] == os.path.split(i[1])[1]):
        key = img_show(i[0],i[1],model)
        if(key == 1):
            break

## Video show example

In [7]:
video_file = 'C:/Users/s_wnsgk4041/20220318_151355.mp4'

video_show(video_file, model)